# Quadratic Constraints

This example demonstrates how to create and solve models with quadratic constraints (QCP/QCQP). A quadratic constraint has the form:

$$ x^T Q x + a^T x \leq b $$

Quadratic constraints work seamlessly with coordinates, just like linear constraints.

In [ ]:
import pandas as pd

import linopy

## Problem Setup

We model a resource allocation problem over multiple time periods. For each time $t$, we want to maximize $x_t + 2y_t$ subject to:
- A budget constraint: $x_t + y_t \leq b_t$
- A risk constraint (quadratic): $x_t^2 + y_t^2 \leq r_t$

where the budget $b_t$ and risk limit $r_t$ vary over time.

In [ ]:
m = linopy.Model()

time = pd.Index(range(5), name="time")

x = m.add_variables(lower=0, coords=[time], name="x")
y = m.add_variables(lower=0, coords=[time], name="y")

## Adding Constraints

Linear constraints work as usual. We use a `pd.Series` for the time-varying budget:

In [ ]:
budget = pd.Series([8, 9, 10, 11, 12], index=time)

m.add_constraints(x + y <= budget, name="budget")

Quadratic constraints use `add_quadratic_constraints`. The risk limits also vary over time:

In [ ]:
risk_limit = pd.Series([20, 25, 30, 35, 40], index=time)

m.add_quadratic_constraints(x**2 + y**2, "<=", risk_limit, name="risk")

## Objective and Solve

We maximize the sum over all time periods:

In [ ]:
m.add_objective((x + 2 * y).sum(), sense="max")
m

To retrieve dual values for quadratic constraints with Gurobi, set `QCPDual=1`:

In [ ]:
m.solve(solver_name="gurobi", QCPDual=1)

## Results

In [ ]:
m.solution.to_dataframe().plot(kind="bar", ylabel="Optimal Value", rot=0);

## Inspecting Quadratic Constraints

Quadratic constraints are stored in `m.quadratic_constraints`:

In [ ]:
m.quadratic_constraints

In [ ]:
m.quadratic_constraints["risk"]

Dual values are available via the `.dual` property:

In [ ]:
m.quadratic_constraints["risk"].dual

## Bilinear Terms

Quadratic constraints can also include cross-product terms like $xy$. Note that bilinear constraints are nonconvex and require a solver that supports them (e.g., Gurobi):

In [ ]:
m2 = linopy.Model()

x = m2.add_variables(lower=0, name="x")
y = m2.add_variables(lower=0, name="y")

# Bilinear constraint: xy <= 4
m2.add_quadratic_constraints(x * y, "<=", 4, name="bilinear")

m2.add_objective(x + y, sense="max")
m2.add_constraints(x <= 5)
m2.add_constraints(y <= 5)

m2.solve(solver_name="gurobi")

print(f"x = {float(x.solution):.2f}, y = {float(y.solution):.2f}")